# Title

# Wald Distribution Intuition

In [4]:
import numpy as np
from ipywidgets import interact
import matplotlib.pyplot as plt

def wald(x, alpha, gamma, theta):
    first = alpha / (np.sqrt(2 * np.pi * np.power(x - theta, 3)) + 1e-6)
    second = np.exp(-(np.power(alpha - gamma * (x - theta), 2)) / (((2 * (x - theta))) + 1e-6))
    return first * second

def plot_wald(db, dr, ndt):
    x = np.arange(ndt, 1.75, .001, dtype=np.float64)
    plt.plot(x, wald(x, db, dr, ndt))
    plt.xlim((0, 1.75))
    plt.show()
    
interact(plot_wald, db=(0, 5, .01), dr=(0, 8, .01), ndt=(0, 2, .01));

interactive(children=(FloatSlider(value=2.0, description=u'db', max=5.0, step=0.01), FloatSlider(value=4.0, description=u'dr', max=8.0, step=0.01), FloatSlider(value=1.0, description=u'ndt', max=2.0, step=0.01), Output()), _dom_classes=('widget-interact',))

# Model Structure

# Prior Determination

# Model Fitting

In [1]:
import pandas as pd
import sys
sys.path.append('../src')
from utils import select_subjects

# load in behavioral data and remove bad trials
data = pd.read_csv('../data/derivatives/behavior/group_data.tsv', sep='\t', 
                   na_values='n/a')
exclusions = ['no_response', 'error', 'post_error', 'fast_rt']
data = data[data[exclusions].sum(axis=1) == 0]

# exclude subjects
subjects = select_subjects('both')
data = data[data.participant_id.isin(subjects)]

# extract data for model
data_in = {}
data_in['Nt'] = data.shape[0]
data_in['rt'] = data.response_time
data_in['Ns']  = len(data.participant_id.unique())
data_in['min_rt'] = data.groupby(['participant_id']).response_time.min().as_matrix() - .001
data_in['sub_ix']= data.participant_id.astype('category').cat.codes + 1
data_in['tt']= data.trial_type.astype('category').cat.codes
data_in['mod']= data.modality.astype('category').cat.codes

In [2]:
import pickle
import os
import pystan

model_name = 'wald_hierarchical'

model_f = '../models/%s.stan' % model_name
cache_f = '../models/%s.pkl' % model_name

if not os.path.exists(cache_f):
    model = pystan.StanModel(file=model_f, model_name=model_name)
    pickle.dump(model, open(cache_f, 'w+'))
else:
    model = pickle.load(open(cache_f, 'r'))

In [7]:
niter = 3000
nwarmup = 2000
nchains = 3

fit = model.sampling(data=data_in, iter=niter, warmup=nwarmup,
                     chains=nchains, init=0, seed=13, thin=5)

# samples = extract_samples(fit, sample_exclude)
# samples.to_csv('%s/%s_samples.csv' % (model_dir, mod), index=False)

# summary = extract_summary(fit, summary_exclude)
# summary.to_csv('%s/%s_summary.csv' % (model_dir, mod), index=True)

In [4]:
import sys
sys.path.append('../src')
from stan_utils import extract_summary, extract_samples

sample_exclude = ['mu', 'lambda', 'db', 'dr', 'ndt', 'ndt_tmp', 'ndt_sub_tmp']
summary_exclude = sample_exclude + ['rt_pred', 'log_lik']

samples = extract_samples(fit, sample_exclude)
samples.to_csv('../models/wald_hierarchical_samples.csv', index=False)

summary = extract_summary(fit, summary_exclude)
summary.to_csv('../models/wald_hierarchical_summary.csv', index=True)

# Model Diagnostics

In [8]:
import sys
sys.path.append('../src')
from stan_utils import param_plot
from ipywidgets import interact
import pandas as pd


params = [p for p in samples.columns.values 
          if 'dr' in p or 'db' in p or 'ndt' in p]

def view_param(param): 
    
    fig = param_plot(param, samples, summary)
    fig.show()

interact(view_param, param=params)

interactive(children=(Dropdown(description=u'param', options=(u'db_beta_mod[0]', u'db_beta_mod[10]', u'db_beta_mod[11]', u'db_beta_mod[12]', u'db_beta_mod[13]', u'db_beta_mod[14]', u'db_beta_mod[15]', u'db_beta_mod[16]', u'db_beta_mod[17]', u'db_beta_mod[18]', u'db_beta_mod[19]', u'db_beta_mod[1]', u'db_beta_mod[20]', u'db_beta_mod[21]', u'db_beta_mod[22]', u'db_beta_mod[23]', u'db_beta_mod[24]', u'db_beta_mod[25]', u'db_beta_mod[26]', u'db_beta_mod[27]', u'db_beta_mod[28]', u'db_beta_mod[29]', u'db_beta_mod[2]', u'db_beta_mod[30]', u'db_beta_mod[31]', u'db_beta_mod[32]', u'db_beta_mod[33]', u'db_beta_mod[34]', u'db_beta_mod[35]', u'db_beta_mod[36]', u'db_beta_mod[37]', u'db_beta_mod[38]', u'db_beta_mod[39]', u'db_beta_mod[3]', u'db_beta_mod[40]', u'db_beta_mod[41]', u'db_beta_mod[42]', u'db_beta_mod[43]', u'db_beta_mod[44]', u'db_beta_mod[45]', u'db_beta_mod[4]', u'db_beta_mod[5]', u'db_beta_mod[6]', u'db_beta_mod[7]', u'db_beta_mod[8]', u'db_beta_mod[9]', u'db_beta_tt[0]', u'db_beta_tt[10]', u'db_beta_tt[11]', u'db_beta_tt[12]', u'db_beta_tt[13]', u'db_beta_tt[14]', u'db_beta_tt[15]', u'db_beta_tt[16]', u'db_beta_tt[17]', u'db_beta_tt[18]', u'db_beta_tt[19]', u'db_beta_tt[1]', u'db_beta_tt[20]', u'db_beta_tt[21]', u'db_beta_tt[22]', u'db_beta_tt[23]', u'db_beta_tt[24]', u'db_beta_tt[25]', u'db_beta_tt[26]', u'db_beta_tt[27]', u'db_beta_tt[28]', u'db_beta_tt[29]', u'db_beta_tt[2]', u'db_beta_tt[30]', u'db_beta_tt[31]', u'db_beta_tt[32]', u'db_beta_tt[33]', u'db_beta_tt[34]', u'db_beta_tt[35]', u'db_beta_tt[36]', u'db_beta_tt[37]', u'db_beta_tt[38]', u'db_beta_tt[39]', u'db_beta_tt[3]', u'db_beta_tt[40]', u'db_beta_tt[41]', u'db_beta_tt[42]', u'db_beta_tt[43]', u'db_beta_tt[44]', u'db_beta_tt[45]', u'db_beta_tt[4]', u'db_beta_tt[5]', u'db_beta_tt[6]', u'db_beta_tt[7]', u'db_beta_tt[8]', u'db_beta_tt[9]', u'db_group_mu_beta_mod', u'db_group_mu_beta_tt', u'db_group_mu_intercept', u'db_group_sigma_beta_mod', u'db_group_sigma_beta_tt', u'db_group_sigma_intercept', u'db_intercept[0]', u'db_intercept[10]', u'db_intercept[11]', u'db_intercept[12]', u'db_intercept[13]', u'db_intercept[14]', u'db_intercept[15]', u'db_intercept[16]', u'db_intercept[17]', u'db_intercept[18]', u'db_intercept[19]', u'db_intercept[1]', u'db_intercept[20]', u'db_intercept[21]', u'db_intercept[22]', u'db_intercept[23]', u'db_intercept[24]', u'db_intercept[25]', u'db_intercept[26]', u'db_intercept[27]', u'db_intercept[28]', u'db_intercept[29]', u'db_intercept[2]', u'db_intercept[30]', u'db_intercept[31]', u'db_intercept[32]', u'db_intercept[33]', u'db_intercept[34]', u'db_intercept[35]', u'db_intercept[36]', u'db_intercept[37]', u'db_intercept[38]', u'db_intercept[39]', u'db_intercept[3]', u'db_intercept[40]', u'db_intercept[41]', u'db_intercept[42]', u'db_intercept[43]', u'db_intercept[44]', u'db_intercept[45]', u'db_intercept[4]', u'db_intercept[5]', u'db_intercept[6]', u'db_intercept[7]', u'db_intercept[8]', u'db_intercept[9]', u'dr_beta_mod[0]', u'dr_beta_mod[10]', u'dr_beta_mod[11]', u'dr_beta_mod[12]', u'dr_beta_mod[13]', u'dr_beta_mod[14]', u'dr_beta_mod[15]', u'dr_beta_mod[16]', u'dr_beta_mod[17]', u'dr_beta_mod[18]', u'dr_beta_mod[19]', u'dr_beta_mod[1]', u'dr_beta_mod[20]', u'dr_beta_mod[21]', u'dr_beta_mod[22]', u'dr_beta_mod[23]', u'dr_beta_mod[24]', u'dr_beta_mod[25]', u'dr_beta_mod[26]', u'dr_beta_mod[27]', u'dr_beta_mod[28]', u'dr_beta_mod[29]', u'dr_beta_mod[2]', u'dr_beta_mod[30]', u'dr_beta_mod[31]', u'dr_beta_mod[32]', u'dr_beta_mod[33]', u'dr_beta_mod[34]', u'dr_beta_mod[35]', u'dr_beta_mod[36]', u'dr_beta_mod[37]', u'dr_beta_mod[38]', u'dr_beta_mod[39]', u'dr_beta_mod[3]', u'dr_beta_mod[40]', u'dr_beta_mod[41]', u'dr_beta_mod[42]', u'dr_beta_mod[43]', u'dr_beta_mod[44]', u'dr_beta_mod[45]', u'dr_beta_mod[4]', u'dr_beta_mod[5]', u'dr_beta_mod[6]', u'dr_beta_mod[7]', u'dr_beta_mod[8]', u'dr_beta_mod[9]', u'dr_beta_tt[0]', u'dr_beta_tt[10]', u'dr_beta_tt[11]', u'dr_beta_tt[12]', u'dr_beta_tt[13]', u'dr_beta_tt[14]', u'dr_beta_tt[15]', u'dr_be

<function __main__.view_param>